In [3]:
#Training the image classifier
import torch
import torchvision
import torchvision.transforms as transforms
from Bio.Seq import Seq
import os
import glob
import pathlib
import pickle
import numpy as np


In [4]:
strep_train = pickle.load(open('C:\\Users\\patri\\Desktop\\pnnl single position here\\training data\\strep_train.txt', 'rb'))
lacto_train = pickle.load(open('C:\\Users\\patri\\Desktop\\pnnl single position here\\training data\\lacto_train.txt', 'rb'))
bac_train = pickle.load(open('C:\\Users\\patri\\Desktop\\pnnl single position here\\training data\\bac_train.txt', 'rb'))
pseudo_train = pickle.load(open('C:\\Users\\patri\\Desktop\\pnnl single position here\\training data\\pseudo_train.txt', 'rb'))

strep_train_array = np.asarray(strep_train)
strep_train_array = np.reshape(strep_train_array, [len(strep_train_array), 1, 64, 64])
# print(strep_train_array.shape)

lacto_train_array = np.asarray(lacto_train)
lacto_train_array = np.reshape(lacto_train_array, [len(lacto_train_array), 1, 64, 64])
# print(lacto_train_array.shape)

bac_train_array = np.asarray(bac_train)
bac_train_array = np.reshape(bac_train_array, [len(bac_train_array), 1, 64, 64])
# print(bac_train_array.shape)

pseudo_train_array = np.asarray(pseudo_train)
pseudo_train_array = np.reshape(pseudo_train_array, [len(pseudo_train_array), 1, 64, 64])
# print(pseudo_train_array.shape)

train_full_list = np.concatenate((strep_train_array, lacto_train_array, bac_train_array, pseudo_train_array))

print(train_full_list.shape)

print(strep_train_array[5])

print(train_full_list[5])

label_bac = np.ones(len(bac_train_array)) * 1
label_lacto = np.ones(len(lacto_train_array)) * 2
label_pseudo = np.ones(len(pseudo_train_array)) * 3
label_strep = np.ones(len(strep_train_array)) * 4

train_label_list = np.concatenate((label_strep, label_lacto, label_bac, label_pseudo))

print(len(train_label_list))

(351744, 1, 64, 64)
[[[15 15 14 ...  4  0  6]
  [13  6 13 ...  4  2 14]
  [10 12  3 ... 11 11  9]
  ...
  [13  7  5 ...  7  4  1]
  [11  9  6 ...  4  1  8]
  [ 3  3  0 ... 13  4  3]]]
[[[15 15 14 ...  4  0  6]
  [13  6 13 ...  4  2 14]
  [10 12  3 ... 11 11  9]
  ...
  [13  7  5 ...  7  4  1]
  [11  9  6 ...  4  1  8]
  [ 3  3  0 ... 13  4  3]]]
351744


In [5]:
strep_test = pickle.load(open('C:\\Users\\patri\\Desktop\\pnnl single position here\\testing data\\strep_test.txt', 'rb'))
lacto_test = pickle.load(open('C:\\Users\\patri\\Desktop\\pnnl single position here\\testing data\\lacto_test.txt', 'rb'))
bac_test = pickle.load(open('C:\\Users\\patri\\Desktop\\pnnl single position here\\testing data\\bac_test.txt', 'rb'))
pseudo_test = pickle.load(open('C:\\Users\\patri\\Desktop\\pnnl single position here\\testing data\\pseudo_test.txt', 'rb'))

strep_test_array = np.asarray(strep_test)
strep_test_array = np.reshape(strep_test_array, [len(strep_test_array), 1, 64, 64])
print(strep_test_array.shape)

lacto_test_array = np.asarray(lacto_test)
lacto_test_array = np.reshape(lacto_test_array, [len(lacto_test_array), 1, 64, 64])
print(lacto_test_array.shape)

bac_test_array = np.asarray(bac_test)
bac_test_array = np.reshape(bac_test_array, [len(bac_test_array), 1, 64, 64])
print(bac_test_array.shape)

pseudo_test_array = np.asarray(pseudo_test)
pseudo_test_array = np.reshape(pseudo_test_array, [len(pseudo_test_array), 1, 64, 64])
print(pseudo_test_array.shape)

test_full_list = np.concatenate((strep_test_array, lacto_test_array, bac_test_array, pseudo_test_array))

# print(test_full_list.shape)

# print(strep_test_array[5])

# print(test_full_list[5])

label_bac = np.ones(len(bac_test_array)) * 1
label_lacto = np.ones(len(lacto_test_array)) * 2
label_pseudo = np.ones(len(pseudo_test_array)) * 3
label_strep = np.ones(len(strep_test_array)) * 4

test_label_list = np.concatenate((label_strep, label_lacto, label_bac, label_pseudo))

(5737, 1, 64, 64)
(1873, 1, 64, 64)
(15115, 1, 64, 64)
(17877, 1, 64, 64)
(40602, 1, 64, 64)
[[[13  6 12 ...  9  7  6]
  [15 12  3 ...  4  2 12]
  [ 3  2 12 ...  3  0  5]
  ...
  [ 8  1  9 ...  0  6 15]
  [15 14 10 ...  6 15 15]
  [15 15 13 ...  7  4  1]]]
[[[13  6 12 ...  9  7  6]
  [15 12  3 ...  4  2 12]
  [ 3  2 12 ...  3  0  5]
  ...
  [ 8  1  9 ...  0  6 15]
  [15 14 10 ...  6 15 15]
  [15 15 13 ...  7  4  1]]]


In [13]:
# training dataset assembly
from torch.utils.data import TensorDataset, DataLoader

# use x_train and y_train as numpy array without further modification
x_train = np.array(train_full_list)
y_train = np.array(train_label_list)

# convert to numpys to tensor
tensor_x = torch.Tensor(x_train) 
tensor_y = torch.Tensor(y_train)
# create the dataset
custom_dataset = TensorDataset(tensor_x,tensor_y) 
# create your dataloader
my_dataloader = DataLoader(custom_dataset,batch_size=1) 

#check if you can get the desired things
i1, l1 = next(iter(my_dataloader))
print(i1.shape)   # torch.Size([1, 1, 28, 28]) 
print(l1.shape)   # torch.Size([1]) 


MemoryError: Unable to allocate 5.37 GiB for an array with shape (351744, 1, 64, 64) and data type int32

In [1]:
# fix to custom dataset work?
from torch.utils.data import TensorDataset, DataLoader

# use x_train and y_train as numpy array without further modification
x_test = np.array(test_full_list)
y_test = np.array(test_label_list)

# convert to numpys to tensor
tensor_x1 = torch.Tensor(x_test) 
tensor_y1 = torch.Tensor(y_test)
# create the dataset
custom_test_dataset = TensorDataset(tensor_x1,tensor_y1) 
# create your dataloader
my_test_dataloader = DataLoader(custom_dataset,batch_size=1) 

#check if you can get the desired things
i2, l2 = next(iter(my_test_dataloader))
print(i2.shape)   # torch.Size([1, 1, 28, 28]) 
print(l2.shape)   # torch.Size([1]) 

NameError: name 'np' is not defined

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64

trainset = torchvision.datasets.custom_dataset(custom_dataset, train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.custom_dataset(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('streptococcus', 'bacillus', 'lactobacaillus', 'pseudomonas')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# shows image, likely unneeded
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH))

In [ ]:
outputs = net(images)

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

In [ ]:
net.to(device)

In [ ]:
inputs, labels = data[0].to(device), data[1].to(device)